In [1]:
import pandas as pd
import numpy as np
import itertools
import datetime
import pandas_gbq
import matplotlib.pyplot as plt
from datetime import *
from datetime import datetime, timedelta, date
# %load_ext google.colab.data_table
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Standard plotly imports
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
#Change the date to the previous day
query = """
SELECT
*
FROM
`perceptive-ivy-290216.f1_api.results_race`
# WHERE Year=2024 AND GP="Austrian Grand Prix"
"""
project_id = "perceptive-ivy-290216"
df_bq2 = pandas_gbq.read_gbq(query, project_id=project_id, dialect='standard')


Downloading: 100%|██████████|


In [4]:
GP=df_bq2
GP=GP.fillna(0)
GP["Position"]=GP["Position"].astype(int)
GP["GridPosition"]=GP["GridPosition"].astype(int)
GP["Points"]=GP["Points"].astype(int)
GP.head()

,DriverNumber,BroadcastName,Abbreviation,DriverId,TeamName,TeamColor,TeamId,FirstName,LastName,FullName,HeadshotUrl,CountryCode,Position,ClassifiedPosition,GridPosition,Q1,Q2,Q3,Time,Status,Points,Year,GP
0,23,A ALBON,ALB,ALB,Williams,37bedd,null,Alexander,Albon,Alexander Albon,0,0,0,0,0,NaT,NaT,NaT,0,0,0,2022,Italian Grand Prix
1,2,,,,Brabham,,0,Teo,Fabi,Teo Fabi,0,0,4,0,7,NaT,NaT,NaT,0 days 01:22:09.163000,Finished,3,1984,Austrian Grand Prix
2,2,,,,Brabham,,0,Teo,Fabi,Teo Fabi,0,0,3,0,23,NaT,NaT,NaT,0 days 01:57:08.370000,Finished,4,1984,Detroit Grand Prix
3,71,,,,Kurtis Kraft,,0,Al,Herman,Al Herman,0,0,7,0,16,NaT,NaT,NaT,0 days 04:00:23.770000,Finished,0,1955,Indianapolis 500
4,89,,,,Dunn,,0,Al,Herman,Al Herman,0,0,13,0,23,NaT,NaT,NaT,0 days 03:47:29.600000,Finished,0,1959,Indianapolis 500


In [5]:
#Get the counts by final position in race

In [6]:
grouped = GP.groupby(['TeamName','Position'])["GP"].agg('count').reset_index()
grouped.columns=["Constructor","Position","Count"]
grouped["Rank"] = grouped.groupby(['Position'])["Count"].rank(method="dense", ascending=False)
grouped=grouped.sort_values(by=["Position","Count"],ascending=[True, False])

In [7]:
#Plot 1 lap at a time

In [8]:
grouped2=grouped[(grouped['Position'].isin([1]))&(grouped['Count']>1)]

In [9]:
position=grouped2["Position"].iloc[0]

In [10]:
fig_lap=px.bar(grouped2, 
           x="Constructor", 
           y="Count", 
           color='Constructor',
        #    facet_col='Position', facet_col_wrap=1, 
           height=800,
           width=1400,
           template='simple_white',
           color_discrete_map={
                 "Alpine": "#0093cc",
                 "Aston Martin": "#229971",
                 "Ferrari": "#E80020",
                 "Haas F1 Team": "#B6BABD",
                 "Kick Sauber": "#52e252",
                 "McLaren": "#FF8000",
                 "Mercedes": "#27F4D2",
                 "RB": "#6692FF",
                 "Red Bull Racing": "#3671C6",
                 "Williams": "#64C4FF" ,
                 "Alfa Romeo":"#C92D4B",
                 "AlphaTauri":"#5E8FAA",
                 "Racing Point":"#F596C8",
                 "Renault":"#FFF500",
                 "Toro Rosso":"#469bff",
                 "Force India":"#F596C8",
                 "Sauber":"#9B0000"
                 },
        )
fig_lap.update_layout(
    hoverlabel=dict(
        bgcolor="white",
        font_size=16,
        font_family="PT Sans Narrow"
    ),
    title="All Time Constructor Race Wins",
    title_x=0.5,
    yaxis = dict(tickfont = dict(size=15)),
    xaxis = dict(tickfont = dict(size=15)),
    font=dict(
        family="PT Sans Narrow",
        size=14,
        color="Black"
    ),
    title_font_family="PT Sans Narrow"
)
# fig_lap.update_traces(marker_color='#246DF8')

In [11]:
fig_lap.write_html("Constructor Race Wins All Time.html",full_html=False, include_plotlyjs='cdn')

In [12]:
#Plot the top in each final position

In [13]:
grouped_top=grouped[grouped["Rank"]==1]
grouped_top=grouped_top[grouped_top["Position"]!=0]

In [14]:
fig_top=px.bar(grouped_top, 
           x="Position", 
           y="Count", 
           color='Constructor',
           height=800,
           width=1400,
           template='simple_white',
           color_continuous_scale=px.colors.sequential.Sunsetdark,
           hover_data=["Constructor"],
           color_discrete_map={
                 "Alpine": "#0093cc",
                 "Aston Martin": "#229971",
                 "Ferrari": "#E80020",
                 "Haas F1 Team": "#B6BABD",
                 "Kick Sauber": "#52e252",
                 "McLaren": "#FF8000",
                 "Mercedes": "#27F4D2",
                 "RB": "#6692FF",
                 "Red Bull Racing": "#3671C6",
                 "Williams": "#64C4FF" ,
                 "Alfa Romeo":"#C92D4B",
                 "AlphaTauri":"#5E8FAA",
                 "Racing Point":"#F596C8",
                 "Renault":"#FFF500",
                 "Toro Rosso":"#469bff",
                 "Force India":"#F596C8",
                 "Sauber":"#9B0000"
                 },
        )

fig_top.update_layout(
    title="Counts By Constructor Final Race Position All Time",
    hoverlabel=dict(
        bgcolor="white",
        font_size=15,
        font_family="PT Sans Narrow"
    ),
    title_x=0.5,
    yaxis = dict(tickfont = dict(size=15)),
    xaxis = dict(tickfont = dict(size=15),
                 tickmode = 'linear',
    tick0 = 1,
    dtick = 1),
    font=dict(
        family="PT Sans Narrow",
        size=15,
        color="Black"
    ),
    title_font_family="PT Sans Narrow"
)
# fig_top.update_traces(marker_color='#246DF8')

In [15]:
fig_top.write_html("Counts By Constructor Final Race Position.html",full_html=False, include_plotlyjs='cdn')

In [16]:
#By Grid Position

In [17]:
grouped_grid = GP.groupby(['TeamName','GridPosition'])["GP"].agg('count').reset_index()
grouped_grid.columns=["TeamName","GridPosition","Count"]
grouped_grid["Rank"] = grouped_grid.groupby(['GridPosition'])["Count"].rank(method="dense", ascending=False)
grouped_grid=grouped_grid.sort_values(by=["GridPosition","Count"],ascending=[True, False])

In [18]:
#Plot 1 lap at a time

In [19]:
grouped2_grid=grouped_grid[(grouped_grid['GridPosition'].isin([1]))&(grouped_grid['Count']>1)]

In [20]:
position=grouped2_grid["GridPosition"].iloc[0]

In [21]:
fig_lap_grid_pos=px.bar(grouped2_grid, 
           x="TeamName", 
           y="Count", 
           color="TeamName",
        #    facet_col='Position', facet_col_wrap=1, 
           height=800,
           width=1400,
           template='simple_white',
           color_discrete_map={
                 "Alpine": "#0093cc",
                 "Aston Martin": "#229971",
                 "Ferrari": "#E80020",
                 "Haas F1 Team": "#B6BABD",
                 "Kick Sauber": "#52e252",
                 "McLaren": "#FF8000",
                 "Mercedes": "#27F4D2",
                 "RB": "#6692FF",
                 "Red Bull Racing": "#3671C6",
                 "Williams": "#64C4FF" ,
                 "Alfa Romeo":"#C92D4B",
                 "AlphaTauri":"#5E8FAA",
                 "Racing Point":"#F596C8",
                 "Renault":"#FFF500",
                 "Toro Rosso":"#469bff",
                 "Force India":"#F596C8",
                 "Sauber":"#9B0000"
                 },
        )
fig_lap_grid_pos.update_layout(
    hoverlabel=dict(
        bgcolor="white",
        font_size=16,
        font_family="PT Sans Narrow"
    ),
    title="All Time #1 Constructor Race Grid Start Position",
    title_x=0.5,
    yaxis = dict(tickfont = dict(size=15)),
    xaxis = dict(tickfont = dict(size=15)),
    font=dict(
        family="PT Sans Narrow",
        size=14,
        color="Black"
    ),
    title_font_family="PT Sans Narrow"
)
# fig_lap_grid_pos.update_traces(marker_color='#246DF8')

In [22]:
fig_lap_grid_pos.write_html("All Time Constructor Grid 1 Start Position.html",full_html=False, include_plotlyjs='cdn')

In [23]:
#Plot the top grid positions

In [24]:
grouped_top_grid=grouped_grid[grouped_grid["Rank"]==1]
grouped_top_grid=grouped_top_grid[grouped_top_grid["GridPosition"]!=0]

In [25]:
fig_top_grid=px.bar(grouped_top_grid, 
           x="GridPosition", 
           y="Count", 
           color='TeamName',
           height=800,
           width=1400,
           template='simple_white',
           hover_data=["TeamName"],
                      color_discrete_map={
                 "Alpine": "#0093cc",
                 "Aston Martin": "#229971",
                 "Ferrari": "#E80020",
                 "Haas F1 Team": "#B6BABD",
                 "Kick Sauber": "#52e252",
                 "McLaren": "#FF8000",
                 "Mercedes": "#27F4D2",
                 "RB": "#6692FF",
                 "Red Bull Racing": "#3671C6",
                 "Williams": "#64C4FF" ,
                 "Alfa Romeo":"#C92D4B",
                 "AlphaTauri":"#5E8FAA",
                 "Racing Point":"#F596C8",
                 "Renault":"#FFF500",
                 "Toro Rosso":"#469bff",
                 "Force India":"#F596C8",
                 "Sauber":"#9B0000"
                 },
        )

fig_top_grid.update_layout(
    title="Counts By Constructor Race Grid Start Position All Time",
    hoverlabel=dict(
        bgcolor="white",
        font_size=15,
        font_family="PT Sans Narrow"
    ),
    title_x=0.5,
    yaxis = dict(tickfont = dict(size=15)),
    xaxis = dict(tickfont = dict(size=15),
                 tickmode = 'linear',
    tick0 = 1,
    dtick = 1),
    font=dict(
        family="PT Sans Narrow",
        size=15,
        color="Black"
    ),
    title_font_family="PT Sans Narrow"
)
# fig_top.update_traces(marker_color='#246DF8')

In [26]:
fig_top_grid.write_html("Counts By Constructor Race Start Grid Position.html",full_html=False, include_plotlyjs='cdn')

In [27]:
#Team Points All Time

In [28]:
Team_all_time=GP.groupby(['TeamName'])["Points"].agg('sum').reset_index().sort_values(by=["Points"],ascending=False)
Team_all_time_filtered=Team_all_time[(Team_all_time['Points']>500)]

In [29]:
fig_team_pts_all_time=px.histogram(Team_all_time_filtered, 
           x="TeamName", 
           y="Points", 
           color='TeamName',
        #    text_auto=True,
        template='simple_white',
           height=800,
           width=1400,
           color_discrete_map={
                 "Alpine": "#0093cc",
                 "Aston Martin": "#229971",
                 "Ferrari": "#E80020",
                 "Haas F1 Team": "#B6BABD",
                 "Kick Sauber": "#52e252",
                 "McLaren": "#FF8000",
                 "Mercedes": "#27F4D2",
                 "RB": "#6692FF",
                 "Red Bull Racing": "#3671C6",
                 "Williams": "#64C4FF" ,
                 "Alfa Romeo":"#C92D4B",
                 "AlphaTauri":"#5E8FAA",
                 "Racing Point":"#F596C8",
                 "Renault":"#FFF500",
                 "Toro Rosso":"#469bff",
                 "Force India":"#F596C8",
                 "Sauber":"#9B0000"
                 },
            hover_data=["TeamName"]
        )

fig_team_pts_all_time.update_layout(
    title="Team Points All Time",
    showlegend=False,    
    hoverlabel=dict(
        bgcolor="white",
        font_size=15,
        font_family="PT Sans Narrow"
    ),
    title_x=0.5,
    yaxis = dict(tickfont = dict(size=15)),
    xaxis = dict(tickfont = dict(size=15)),
    font=dict(
        family="PT Sans Narrow",
        size=15,
        color="Black"
    ),
    title_font_family="PT Sans Narrow"
)

fig_team_pts_all_time.update_xaxes(title_text='')      # Remove axis titles
fig_team_pts_all_time.update_yaxes(title_text='')
# fig_team_pts_all_time.update_traces(textfont_size=12, textposition="outside")


In [30]:
fig_team_pts_all_time.write_html("Constructor Points All Time.html",full_html=False, include_plotlyjs='cdn')

In [31]:
#Qualifying

In [32]:
#Change the date to the previous day
query = """
SELECT
*
FROM
`perceptive-ivy-290216.f1_api.results_qualifying`
# WHERE Year=2024 AND GP="Austrian Grand Prix"
"""
project_id = "perceptive-ivy-290216"
df_bq2_qualy = pandas_gbq.read_gbq(query, project_id=project_id, dialect='standard')


Downloading: 100%|██████████|


In [33]:
GP_qualy=df_bq2_qualy
GP_qualy=GP_qualy.fillna(0)
GP_qualy["Position"]=GP_qualy["Position"].astype(int)
GP_qualy["GridPosition"]=GP_qualy["GridPosition"].astype(int)
GP_qualy["Points"]=GP_qualy["Points"].astype(int)
GP_qualy.head()

,DriverNumber,BroadcastName,Abbreviation,DriverId,TeamName,TeamColor,TeamId,FirstName,LastName,FullName,HeadshotUrl,CountryCode,Position,ClassifiedPosition,GridPosition,Q1,Q2,Q3,Time,Status,Points,Year,GP
0,21,0,ALO,0,Minardi,0,0,Fernando,Alonso,Fernando Alonso,0,0,17,17,17,0 days 00:01:13.991000,0,0,0,0,0,2001,United States Grand Prix
1,24,0,BAD,0,Minardi,0,0,Luca,Badoer,Luca Badoer,0,0,12,12,12,0 days 00:01:20.543000,0,0,0,0,0,1995,Hungarian Grand Prix
2,24,0,BAD,0,Minardi,0,0,Luca,Badoer,Luca Badoer,0,0,13,13,13,0 days 00:01:57.167000,0,0,0,0,0,1995,Argentine Grand Prix
3,24,0,BAD,0,Minardi,0,0,Luca,Badoer,Luca Badoer,0,0,15,15,15,0,0,0,0,0,0,1995,Australian Grand Prix
4,24,0,BAD,0,Minardi,0,0,Luca,Badoer,Luca Badoer,0,0,16,16,16,0 days 00:01:49.302000,0,0,0,0,0,1995,German Grand Prix


In [34]:
#Get the counts by final position in Qualy

In [35]:
grouped_qualy = GP_qualy.groupby(['TeamName','Position'])["GP"].agg('count').reset_index()
grouped_qualy.columns=["TeamName","Position","Count"]
grouped_qualy["Rank"] = grouped_qualy.groupby(['Position'])["Count"].rank(method="dense", ascending=False)
grouped_qualy=grouped_qualy.sort_values(by=["Position","Count"],ascending=[True, False])

In [36]:
#Plot 1 lap at a time

In [37]:
grouped2_qualy=grouped_qualy[(grouped_qualy['Position'].isin([1]))&(grouped_qualy['Count']>0)]

In [38]:
position_qualy=grouped2_qualy["Position"].iloc[0]

In [39]:
fig_lap_qualy=px.bar(grouped2_qualy, 
           x="TeamName", 
           y="Count", 
           color="TeamName",
        #    facet_col='Position', facet_col_wrap=1, 
           height=800,
           width=1400,
           template='simple_white',
           color_discrete_map={
                 "Alpine": "#0093cc",
                 "Aston Martin": "#229971",
                 "Ferrari": "#E80020",
                 "Haas F1 Team": "#B6BABD",
                 "Kick Sauber": "#52e252",
                 "McLaren": "#FF8000",
                 "Mercedes": "#27F4D2",
                 "RB": "#6692FF",
                 "Red Bull Racing": "#3671C6",
                 "Williams": "#64C4FF" ,
                 "Alfa Romeo":"#C92D4B",
                 "AlphaTauri":"#5E8FAA",
                 "Racing Point":"#F596C8",
                 "Renault":"#FFF500",
                 "Toro Rosso":"#469bff",
                 "Force India":"#F596C8",
                 "Sauber":"#9B0000"
                 },
        )
fig_lap_qualy.update_layout(
    hoverlabel=dict(
        bgcolor="white",
        font_size=16,
        font_family="PT Sans Narrow"
    ),
    title="Constructor Qualifying Position #1 All Time",
    title_x=0.5,
    yaxis = dict(tickfont = dict(size=15)),
    xaxis = dict(tickfont = dict(size=15),
                 tickmode = 'linear',
    tick0 = 1,
    dtick = 1),
    font=dict(
        family="PT Sans Narrow",
        size=14,
        color="Black"
    ),
    title_font_family="PT Sans Narrow"
)
# fig_lap_qualy.update_traces(marker_color='#246DF8')

In [40]:
fig_lap_qualy.write_html("Constructor Qualifying Results All Time.html",full_html=False, include_plotlyjs='cdn')

In [41]:
#Plot the top in each final position

In [42]:
grouped_top_qualy=grouped_qualy[grouped_qualy["Rank"]==1]
grouped_top_qualy=grouped_top[grouped_top["Position"]!=0]

In [43]:
fig_top_qualy=px.bar(grouped_top_qualy, 
           x="Position", 
           y="Count", 
           color='Constructor',
           height=800,
           width=1400,
           template='simple_white',
           hover_data=["Constructor"],
            color_discrete_map={
                 "Alpine": "#0093cc",
                 "Aston Martin": "#229971",
                 "Ferrari": "#E80020",
                 "Haas F1 Team": "#B6BABD",
                 "Kick Sauber": "#52e252",
                 "McLaren": "#FF8000",
                 "Mercedes": "#27F4D2",
                 "RB": "#6692FF",
                 "Red Bull Racing": "#3671C6",
                 "Williams": "#64C4FF" ,
                 "Alfa Romeo":"#C92D4B",
                 "AlphaTauri":"#5E8FAA",
                 "Racing Point":"#F596C8",
                 "Renault":"#FFF500",
                 "Toro Rosso":"#469bff",
                 "Force India":"#F596C8",
                 "Sauber":"#9B0000"
                 },
        )

fig_top_qualy.update_layout(
    title="Counts By Constructor Qualifying Position All Time",
    hoverlabel=dict(
        bgcolor="white",
        font_size=15,
        font_family="PT Sans Narrow"
    ),
    title_x=0.5,
    yaxis = dict(tickfont = dict(size=15)),
    xaxis = dict(tickfont = dict(size=15),
                 tickmode = 'linear',
    tick0 = 1,
    dtick = 1),
    font=dict(
        family="PT Sans Narrow",
        size=15,
        color="Black"
    ),
    title_font_family="PT Sans Narrow"
)
# fig_top_qualy.update_traces(marker_color='#246DF8')

In [44]:
fig_top_qualy.write_html("Counts By Constructor Qualifying Position All Time.html",full_html=False, include_plotlyjs='cdn')